In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model


from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
cd /content/drive/My Drive/Colab Notebooks/HAN

/content/drive/My Drive/Colab Notebooks/HAN


In [0]:
# %load char-rnn.py
import numpy as np
import h5py
import pickle
from copy import deepcopy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing import sequence
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers import Conv1D
import tensorflow as tf

from keras.utils import np_utils
from MyNormalizer import token

################# GLOBAL VARIABLES #####################
#Filenames
#TODO: Add to coding conventions that directories are to always end with '/'
Masterdir = '/Download/Sub-word-LSTM/Sub-word-LSTM-master'
Datadir = 'Data/'
Modeldir = 'Models/'
Featuredir = 'Features/'
inputdatasetfilename = 'IIITH_Codemixed.txt'
exp_details = 'new_experiment'

#Data I/O formatting
SEPERATOR = '\t'
DATA_COLUMN = 1
LABEL_COLUMN = 3
LABELS = ['0','1','2'] # 0 -> Negative, 1-> Neutral, 2-> Positive
mapping_char2num = {}
mapping_num2char = {}
MAXLEN = 200

#LSTM Model Parameters
#Embedding
MAX_FEATURES = 0
embedding_size = 128
# Convolution
filter_length = 3
nb_filter = 128
pool_length = 3
# LSTM
lstm_output_size = 128
# Training
batch_size = 128
number_of_epochs = 80
numclasses = 3
test_size = 0.2
MAX_WORD_LENGTH = 7
MAX_WORDS = 20
MAX_NB_CHARS = 1000
EMBEDDING_DIM = 10
VALIDATION_SPLIT = 0.2
########################################################

In [93]:
def par(filename,seperator,labelcol,labels):
  f=open(filename,'r',encoding='utf-8')
  lines=f.read().lower()
  lines=lines.lower().split('\n')[:-1]
  sentences = [l.split(seperator) for l in lines]
  sentences=[s[1] for s in sentences]
  tokenizer = Tokenizer(num_words=MAX_NB_CHARS, char_level=True)
  tokenizer.fit_on_texts(sentences)
  data = np.zeros((len(sentences), MAX_WORDS, MAX_WORD_LENGTH), dtype='int32')
  for i, words in enumerate(sentences):
    for j, word in enumerate(words):
        if j < MAX_WORDS:
            k = 0
            for _, char in enumerate(word):
                try:
                    if k < MAX_WORD_LENGTH:
                        if tokenizer.word_index[char] < MAX_NB_CHARS:
                            data[i, j, k] = tokenizer.word_index[char]
                            k=k+1
                except:
                    None
  char_index = tokenizer.word_index
  lab=[]
  for line in lines:
    line=line.split(seperator)
    if(line[labelcol]==labels[0]):
      lab.append(0)
    if(line[labelcol]==labels[1]):
      lab.append(1)
    if(line[labelcol]==labels[2]):
      lab.append(2)
  lab=np.array(lab)
  indices = np.arange(data.shape[0])
  np.random.shuffle(indices)
  data = data[indices]
  lab = lab[indices]
  
  nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
  
    
  x_train = data[:-nb_validation_samples]
  y_train = lab[:-nb_validation_samples]
  x_val = data[-nb_validation_samples:]
  y_val = lab[-nb_validation_samples:]
  y_train=tf.keras.utils.to_categorical(y_train)
  y_val=tf.keras.utils.to_categorical(y_val)

  return (x_train,y_train,x_val,y_val,char_index)
par(inputdatasetfilename,SEPERATOR,LABEL_COLUMN,LABELS)
  

(array([[[11,  0,  0, ...,  0,  0,  0],
         [16,  0,  0, ...,  0,  0,  0],
         [20,  0,  0, ...,  0,  0,  0],
         ...,
         [23,  0,  0, ...,  0,  0,  0],
         [ 2,  0,  0, ...,  0,  0,  0],
         [ 7,  0,  0, ...,  0,  0,  0]],
 
        [[10,  0,  0, ...,  0,  0,  0],
         [ 2,  0,  0, ...,  0,  0,  0],
         [13,  0,  0, ...,  0,  0,  0],
         ...,
         [ 2,  0,  0, ...,  0,  0,  0],
         [ 1,  0,  0, ...,  0,  0,  0],
         [11,  0,  0, ...,  0,  0,  0]],
 
        [[10,  0,  0, ...,  0,  0,  0],
         [ 2,  0,  0, ...,  0,  0,  0],
         [13,  0,  0, ...,  0,  0,  0],
         ...,
         [ 1,  0,  0, ...,  0,  0,  0],
         [19,  0,  0, ...,  0,  0,  0],
         [16,  0,  0, ...,  0,  0,  0]],
 
        ...,
 
        [[14,  0,  0, ...,  0,  0,  0],
         [ 4,  0,  0, ...,  0,  0,  0],
         [ 2,  0,  0, ...,  0,  0,  0],
         ...,
         [ 8,  0,  0, ...,  0,  0,  0],
         [11,  0,  0, ...,  0,  0,  0],


In [94]:
    print('Starting RNN Engine...\nModel: Char-level LSTM.\nParsing data files...')
    out = par(inputdatasetfilename,SEPERATOR,LABEL_COLUMN,LABELS)
    global x_train
    global y_train
    global x_test
    global y_test
    x_train = out[0]
    y_train = out[1]
    x_val=out[2]
    y_val=out[3]
    char_index=out[4]



Starting RNN Engine...
Model: Char-level LSTM.
Parsing data files...


In [87]:
print(x_train)

[[[14  0  0 ...  0  0  0]
  [ 4  0  0 ...  0  0  0]
  [ 2  0  0 ...  0  0  0]
  ...
  [ 1  0  0 ...  0  0  0]
  [11  0  0 ...  0  0  0]
  [ 8  0  0 ...  0  0  0]]

 [[12  0  0 ...  0  0  0]
  [16  0  0 ...  0  0  0]
  [11  0  0 ...  0  0  0]
  ...
  [ 4  0  0 ...  0  0  0]
  [ 8  0  0 ...  0  0  0]
  [ 1  0  0 ...  0  0  0]]

 [[23  0  0 ...  0  0  0]
  [ 2  0  0 ...  0  0  0]
  [22  0  0 ...  0  0  0]
  ...
  [22  0  0 ...  0  0  0]
  [ 4  0  0 ...  0  0  0]
  [ 2  0  0 ...  0  0  0]]

 ...

 [[ 4  0  0 ...  0  0  0]
  [13  0  0 ...  0  0  0]
  [17  0  0 ...  0  0  0]
  ...
  [25  0  0 ...  0  0  0]
  [ 1  0  0 ...  0  0  0]
  [14  0  0 ...  0  0  0]]

 [[10  0  0 ...  0  0  0]
  [ 2  0  0 ...  0  0  0]
  [13  0  0 ...  0  0  0]
  ...
  [20  0  0 ...  0  0  0]
  [ 1  0  0 ...  0  0  0]
  [20  0  0 ...  0  0  0]]

 [[12  0  0 ...  0  0  0]
  [16  0  0 ...  0  0  0]
  [11  0  0 ...  0  0  0]
  ...
  [ 2  0  0 ...  0  0  0]
  [17  0  0 ...  0  0  0]
  [ 6  0  0 ...  0  0  0]]]


In [86]:
x_train.shape

(3104, 20, 7)

In [0]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [110]:
embedding_layer = Embedding(len(char_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_WORD_LENGTH,
                            trainable=True)

char_input = Input(shape=(MAX_WORD_LENGTH,), dtype='int32')
char_sequences = embedding_layer(char_input)
char_conv=Convolution1D(nb_filter=nb_filter,
							filter_length=filter_length,
							border_mode='valid',
							activation='relu',
							subsample_length=1)(char_sequences)
char_maxpool=MaxPooling1D(pool_length=pool_length)(char_conv)
char_lstm = Bidirectional(LSTM(100, return_sequences=True))(char_maxpool)
char_dense = TimeDistributed(Dense(200))(char_lstm)
char_att = AttentionWithContext()(char_dense)
charEncoder = Model(char_input, char_att)

words_input = Input(shape=(MAX_WORDS, MAX_WORD_LENGTH), dtype='int32')
words_encoder = TimeDistributed(charEncoder)(words_input)
words_lstm = Bidirectional(LSTM(100, return_sequences=True))(words_encoder)
words_dense = TimeDistributed(Dense(200))(words_lstm)
words_att = AttentionWithContext()(words_dense)
preds = Dense(3, activation='softmax')(words_att)
model = Model(words_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['acc'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3, strides=1, padding="valid")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=3)`
  del sys.path[0]


In [112]:
RocAuc = RocAucEvaluation(validation_data=(x_val, y_val), interval=1)
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=100, batch_size=64)

Train on 3104 samples, validate on 775 samples
Epoch 1/100
3104/3104 [==============================] - 11s 4ms/step - loss: 0.9942 - acc: 0.5045 - val_loss: 1.0010 - val_acc: 0.5045
Epoch 2/100
3104/3104 [==============================] - 11s 4ms/step - loss: 0.9930 - acc: 0.5045 - val_loss: 1.0014 - val_acc: 0.5045
Epoch 3/100
3104/3104 [==============================] - 11s 4ms/step - loss: 0.9930 - acc: 0.5045 - val_loss: 0.9987 - val_acc: 0.5045
Epoch 4/100
3104/3104 [==============================] - 11s 4ms/step - loss: 0.9866 - acc: 0.5045 - val_loss: 0.9878 - val_acc: 0.5045
Epoch 5/100
3104/3104 [==============================] - 11s 4ms/step - loss: 0.9757 - acc: 0.5074 - val_loss: 0.9765 - val_acc: 0.5097
Epoch 6/100
3104/3104 [==============================] - 11s 4ms/step - loss: 0.9630 - acc: 0.5151 - val_loss: 0.9680 - val_acc: 0.5252
Epoch 7/100
3104/3104 [==============================] - 11s 4ms/step - loss: 0.9553 - acc: 0.5338 - val_loss: 0.9671 - val_acc: 0.5484
E